In [26]:
import sys
sys.path.append("../..")

In [27]:
from src.marks_reader import *
from src.competition_data import *
from src.prediction_saver import *
from src.ponderator import *
from src.simulator import *
from src.utils import *
from data.events_data import events
from datetime import datetime

In [28]:
events_params = load_json('opt_results4_new.json')
entry_list = load_json('data/entry_list.json')

In [29]:
competition = CompetitionData(
    name="World Championships in Athletics Oregon 2022", 
    events_data=events, 
    events_params=events_params,
    start_date=datetime(2022, 7, 15),
    entry_list=entry_list,
)

event_folder = 'data/classified'
only = [
    'atl_800m',
    'atl_tri',
    'atl_20km',
    'atl_mar'
]
top = 16

In [30]:
data = read_marks(
    folder_path=event_folder, 
    only=only,
    limit_date=competition.start_date
)

In [31]:
ponderated_data = ponderate_all_events(
    data=data, 
    competition=competition,
    years_weight={
        "2020": 1,
        "2021": 2,
        "2022": 4
    },   
    logs=True,
)

In [32]:
prediction = simulate_all_events(
    data=ponderated_data, 
    competition=competition, 
    top=top, 
    times=5000, 
    models_folder='models',
    override_models=True,
    only=only,
    logs=True, 
)

Creating athletes models for event atl_800m and sex male...
(2.00 %) Creating kde model of ALG_djamel_sedjati...
(4.00 %) Creating kde model of ALG_slimane_moula...
(6.00 %) Creating kde model of ALG_yassine_hethat...
(8.00 %) Creating kde model of AUS_joseph_deng...
(10.00 %) Creating kde model of AUS_peter_bol...
(12.00 %) Creating kde model of BEL_eliott_crestan...
(14.00 %) Creating kde model of BIH_abedin_mujezinović...
(16.00 %) Creating kde model of BOT_nijel_amos...
(18.00 %) Creating kde model of BRA_thiago_andré...
(20.00 %) Creating kde model of CAN_brandon_mcbride...
(22.00 %) Creating kde model of CAN_marco_arop...
(24.00 %) Creating kde model of ESP_adrián_ben...
(26.00 %) Creating kde model of ESP_mariano_garcía...
(28.00 %) Creating kde model of ESP_saúl_ordóñez...
(30.00 %) Creating kde model of ESP_álvaro_de_arriba...
(32.00 %) Creating kde model of ETH_ermias_girma...
(34.00 %) Creating kde model of ETH_tolesa_bodena...
(36.00 %) Creating kde model of FRA_benjamin_ro

In [33]:
save_prediction(
    results=prediction, 
    competition=competition, 
    top=top,
    file='predictions444_some.json'
)

In [34]:
# pred = load_json('predictions44_both.json')

# for event in pred:
#     for sex in pred[event]['sex']:
#         d = pred[event]['sex'][sex]['prediction']
#         for k, v in list(d.items()):
#             if int(k) > 8:
#                 del d[k]
        
# save_json(pred, 'predictions44_8both.json')

In [35]:
from src.stats_calculator import *

calculate_all_stats('data/results.json', 'predictions444_some.json', 'stats444_some.json')

In [36]:
# stats = load_json('stats4_both.json')

# p_total_ranking_pos = stats['total_ranking_pos'] / 336 * 100
# p_total_ranking_exact_pos = stats['total_ranking_exact_pos'] / 336 * 100
# p_total_podium_pos = stats['total_podium_pos'] / 126 * 100
# p_total_podium_exact_pos = stats['total_podium_exact_pos'] / 126 * 100
# p_total_champion = stats['total_champion'] / 42 * 100

# p_total_ranking_pos = round(p_total_ranking_pos, 2)
# p_total_ranking_exact_pos = round(p_total_ranking_exact_pos, 2)
# p_total_podium_pos = round(p_total_podium_pos, 2)
# p_total_podium_exact_pos = round(p_total_podium_exact_pos, 2)
# p_total_champion = round(p_total_champion, 2)

# print(p_total_ranking_pos, p_total_ranking_exact_pos, p_total_podium_pos, p_total_podium_exact_pos, p_total_champion)

In [37]:
# stats = load_json('stats44_new.json')

# p_total_ranking_pos = stats['total_ranking_pos'] / 336 * 100
# p_total_ranking_exact_pos = stats['total_ranking_exact_pos'] / 336 * 100
# p_total_podium_pos = stats['total_podium_pos'] / 126 * 100
# p_total_podium_exact_pos = stats['total_podium_exact_pos'] / 126 * 100
# p_total_champion = stats['total_champion'] / 42 * 100

# p_total_ranking_pos = round(p_total_ranking_pos, 2)
# p_total_ranking_exact_pos = round(p_total_ranking_exact_pos, 2)
# p_total_podium_pos = round(p_total_podium_pos, 2)
# p_total_podium_exact_pos = round(p_total_podium_exact_pos, 2)
# p_total_champion = round(p_total_champion, 2)

# print(p_total_ranking_pos, p_total_ranking_exact_pos, p_total_podium_pos, p_total_podium_exact_pos, p_total_champion)

In [38]:
# p = ""
# stats = load_json('stats4_new.json')

# for event in stats:
#     try:
#         try:
#             ranking_pos_male = stats[event]['male']['ranking_pos']
#             ranking_exact_pos_male = stats[event]['male']['ranking_exact_pos']
#             podium_pos_male = stats[event]['male']['podium_pos']
#             podium_exact_pos_male = stats[event]['male']['podium_exact_pos']
#             champion_male = stats[event]['male']['champion']
#         except:
#             ranking_pos_male = '-'
#             ranking_exact_pos_male = '-'
#             podium_pos_male = '-'
#             podium_exact_pos_male = '-'
#             champion_male = '-'
            
#         try:
#             ranking_pos_female = stats[event]['female']['ranking_pos']
#             ranking_exact_pos_female = stats[event]['female']['ranking_exact_pos']
#             podium_pos_female = stats[event]['female']['podium_pos']
#             podium_exact_pos_female = stats[event]['female']['podium_exact_pos']
#             champion_female = stats[event]['female']['champion']
#         except:
#             ranking_pos_female = '-'
#             ranking_exact_pos_female = '-'
#             podium_pos_female = '-'
#             podium_exact_pos_female = '-'
#             champion_female = '-'

#         p += f"           {competition.get_event_data(event,'name')} & {ranking_pos_female} & {ranking_pos_male} & {ranking_exact_pos_female} & {ranking_exact_pos_male} & {podium_pos_female} & {podium_pos_male} & {podium_exact_pos_female} & {podium_exact_pos_male} & {champion_female} & {champion_male} \\\\\n"

#     except:
#         continue
# p+= '           \hline\n'
# p += f"           Total & {stats['total_ranking_pos_gender']['female']} & {stats['total_ranking_pos_gender']['male']} & {stats['total_ranking_exact_pos_gender']['female']} & {stats['total_ranking_exact_pos_gender']['male']} & {stats['total_podium_pos_gender']['female']} & {stats['total_podium_pos_gender']['male']} & {stats['total_podium_exact_pos_gender']['female']} & {stats['total_podium_exact_pos_gender']['male']} & {stats['total_champion_gender']['female']} & {stats['total_champion_gender']['male']} \\\\ \hline"


# print(p)

 

In [39]:
# p = ""
# stats = load_json('expert_params.json')

# for event in stats:
#     try:
#         alpha_female = round(stats[event]['female']['alpha'], 2)
#         bandwidth_female = round(stats[event]['female']['bandwidth'], 2)
#         times_female = stats[event]['female']['sim_times']
#         # pond_female = stats[event]['female']['pond_times']
#         # pond_female = f"{pond_female['2022']} {pond_female['2021']} {pond_female['2020']}"
#         pond_female = '4 2 1'
#     except:
#         alpha_female = '-'
#         bandwidth_female = '-'
#         times_female = '-'
#         pond_female = '-'
#     try:
#         alpha_male = round(stats[event]['male']['alpha'], 2)
#         bandwidth_male = round(stats[event]['male']['bandwidth'], 2)
#         times_male = stats[event]['male']['sim_times']
#         # pond_male = stats[event]['male']['pond_times']
#         # pond_male = f"{pond_male['2022']} {pond_male['2021']} {pond_male['2020']}"
#         pond_male = '4 2 1'
#     except:
#         alpha_male = '-'
#         bandwidth_male = '-'
#         times_male = '-'
#         pond_male = '-'

#     try:
#         p += f"           {competition.get_event_data(event,'name')} & {alpha_female} & {alpha_male} & {bandwidth_female} & {bandwidth_male} & {times_female} & {times_male} & {pond_female} & {pond_male} \\\\\n"
#     except:
#         p += f"           Marcha 50 kilómetros & {alpha_female} & {alpha_male} & {bandwidth_female} & {bandwidth_male} & {times_female} & {times_male} & {pond_female} & {pond_male} \\\\\n"


# print(p)

 

In [40]:
# from src.stats_calculator import convert_pred

# results = load_json('results.json')
# predictions = load_json('predictions0.json')
# # predictions = convert_pred(predictions)
    
# count = 0
   
# for event in results:
#     for sex in results[event]:
#         res = list(results[event][sex].values())
#         pred = list(predictions[event][sex].values())
    
#         if res[0] == pred[0]:
#             count += 1

# print(count)




In [41]:
# from src.utils import *

# p = ""
# stats = load_json('stats4_new.json')

# for event in stats:
#     try:
#         ranking_pos = stats[event]['male']['ranking_pos'] + stats[event]['female']['ranking_pos']
#         ranking_exact_pos = stats[event]['male']['ranking_exact_pos'] + stats[event]['female']['ranking_exact_pos']
#         podium_pos = stats[event]['male']['podium_pos'] + stats[event]['female']['podium_pos']
#         podium_exact_pos = stats[event]['male']['podium_exact_pos'] + stats[event]['female']['podium_exact_pos']
#     except:
#         try:
#             ranking_pos = stats[event]['male']['ranking_pos']
#             ranking_exact_pos = stats[event]['male']['ranking_exact_pos']
#             podium_pos = stats[event]['male']['podium_pos']
#             podium_exact_pos = stats[event]['male']['podium_exact_pos']
#         except:
#             try:
#                 ranking_pos = stats[event]['female']['ranking_pos']
#                 ranking_exact_pos = stats[event]['female']['ranking_exact_pos']
#                 podium_pos = stats[event]['female']['podium_pos']
#                 podium_exact_pos = stats[event]['female']['podium_exact_pos']
#             except:
#                 continue

#     p += f"           {competition.get_event_data(event,'name')} & {ranking_pos} & {ranking_exact_pos} & {podium_pos} & {podium_exact_pos} \\\\\n"

# print(p)

In [42]:
# pred = load_json('predictions2.json')

# medals = {}

# for event in pred:
#     for sex in pred[event]['sex']:
#         p = pred[event]['sex'][sex]['prediction']
#         gold = p['1']['country']
#         silver = p['2']['country']
#         bronce = p['3']['country']

#         if gold not in medals:
#             medals[gold] = {}
#         if silver not in medals:
#             medals[silver] = {}
#         if bronce not in medals:
#             medals[bronce] = {}

#         try:
#             medals[gold]['gold'] += 1
#         except:
#             medals[gold]['gold'] = 1 
#         try:
#             medals[silver]['silver'] += 1
#         except:
#             medals[silver]['silver'] = 1 
#         try:
#             medals[bronce]['bronce'] += 1
#         except:
#             medals[bronce]['bronce'] = 1 

# print(medals)


In [43]:
# pred = load_json('results.json')

# medals = {}

# for event in pred:
#     for sex in pred[event]:
#         p = pred[event][sex]
#         gold = p['1'].split('_')[0]
#         silver = p['2'].split('_')[0]
#         bronce = p['3'].split('_')[0]

#         if gold not in medals:
#             medals[gold] = {}
#         if silver not in medals:
#             medals[silver] = {}
#         if bronce not in medals:
#             medals[bronce] = {}

#         try:
#             medals[gold]['gold'] += 1
#         except:
#             medals[gold]['gold'] = 1 
#         try:
#             medals[silver]['silver'] += 1
#         except:
#             medals[silver]['silver'] = 1 
#         try:
#             medals[bronce]['bronce'] += 1
#         except:
#             medals[bronce]['bronce'] = 1 

# print(medals)

In [44]:
# medal = 'bronce'
# only = ['GER', 'POL'] # ['NOR', 'PUR', 'BRA', 'BRN', 'KOR', 'UKR', 'SWE', 'CUB', 'GER', 'BUR', 'VEN', 'SLO', 'POL'] # medals.keys()
# less = ['USA', 'ETH', 'CAN', 'JAM','KEN','NED', 'CHN', 'ESP','JPN', ]

# def getn(countries, medal, only, less):
#     m = 0
#     for c in only:
#         if c in less:
#             continue
#         try:
#             m = max(m, countries[c][medal])
#         except:
#             continue
#     return m

# n = getn(medals, medal, only, less)

# def getc(countries, medal, only, n, less):
#     clist = []
#     for c in only:
#         if c in less:
#             continue
#         try:
#             if countries[c][medal] == n:
#                 clist.append(c)
#         except:
#             continue
#     return clist

# clist = getc(medals, medal, only, n, less)
# print(n)
# print(clist)


In [45]:
# places = ['USA', 'ETH', "JAM", "KEN", "CHN", "AUS", "PER", "POL", "CAN", "JPN"]

# for country in places:
#     try:
#         gold = medals[country]['gold']
#     except:
#         gold = 0
#     try:
#         silver = medals[country]['silver']
#     except:
#         silver = 0
#     try:
#         bronce = medals[country]['bronce']
#     except:
#         bronce = 0
#     print(f"    {country}  & {gold}  &   {silver}   &   {bronce}   &  {gold + silver + bronce}   ")

In [46]:
# from sklearn.neighbors import KernelDensity
# import numpy as np 
# import matplotlib.pyplot as plt

# X = data['atl_800m']['female']['USA_athing_mu'].Result.to_numpy()

# a = X.reshape(-1,1)
# kde = KernelDensity(kernel='gaussian', bandwidth=1.5).fit(a)
# s = np.linspace(min(a),max(a))
# e = kde.score_samples(s.reshape(-1,1))
# plt.plot(s, np.exp(e))

In [47]:
from sklearn.neighbors import KernelDensity
import numpy as np 
import matplotlib.pyplot as plt

bounds = [10.3, 12]

X = data['atl_100m']['female']['JAM_shelly-ann_fraser-pryce'].Result.to_numpy()
a = X.reshape(-1,1)
kde = KernelDensity(kernel='gaussian', bandwidth=0.08).fit(a)
s = np.linspace(np.min(bounds),np.max(bounds))
e = kde.score_samples(s.reshape(-1,1))
plt.plot(s, np.exp(e), label='Shelly-ann Fraser-pryce')


X = data['atl_100m']['female']['JAM_shericka_jackson'].Result.to_numpy()
a = X.reshape(-1,1)
kde = KernelDensity(kernel='gaussian', bandwidth=0.08).fit(a)
s = np.linspace(np.min(bounds),np.max(bounds))
e = kde.score_samples(s.reshape(-1,1))
plt.plot(s, np.exp(e), label='Shericka Jackson')

X = data['atl_100m']['female']['JAM_elaine_thompson-herah'].Result.to_numpy()
a = X.reshape(-1,1)
kde = KernelDensity(kernel='gaussian', bandwidth=0.08).fit(a)
s = np.linspace(np.min(bounds),np.max(bounds))
e = kde.score_samples(s.reshape(-1,1))
plt.plot(s, np.exp(e), label='Elaine Thompson-hera')

plt.legend()

KeyError: 'atl_100m'

In [ ]:
x_train = data['atl_mat']['male']

min = sys.maxsize
max = 0
for _, results in x_train.items():
    min = np.min(results.Result.to_list() + [min])
    max = np.max(results.Result.to_list() + [max])
half_range = (max - min) / 2

print(half_range, half_range/100)


KeyError: 'atl_mat'